In [1]:
import pandas as pd
import json
import seaborn as sns
import numpy as np
import os
import csv
from tqdm.notebook import tqdm
import datetime

# 네이버 메인 파일 수정하기.

In [119]:
'''
네이버 메인 파일 수정.
한 기사에 대해서 여러 날짜에 대해 올라오는 것이 가능하며, 현재는 (제목, 언론사) key로 되어 있어서
제목이 비어 있는 경우 중복으로 묶이게 됨.
따라서 aid 기준으로 mapping되는 새로운 main_info file을 만들고자 함.
'''

from collections import defaultdict
from datetime import datetime

revised_main = defaultdict(list)
def extract_aid (info):
    aid = ','.join([info['url'].split('&')[-2][-3:], info['url'].split('&')[-1][4:]])
    return aid

revised_main = defaultdict(list)
naver_main_path = '../news data/main/main_naver/main/'
naver_main_list = os.listdir(naver_main_path)
naver_main_list_original = naver_main_list
naver_main_list = list(map(lambda x: x[:4] + x[5:7] + x[8:], naver_main_list_original))

for day in tqdm(naver_main_list_original):
    with open(naver_main_path + day, 'r') as f:
        file = json.load(f)
        for info in file:
            aid = extract_aid(info)
            info['date'] = datetime.strptime(day[:10], '%Y-%m-%d')
            revised_main[aid].append(info)

revised_main = {k:v for k,v in sorted(revised_main.items(), key = lambda x: len(x[1]), reverse = True)}

100%|██████████| 3589/3589 [00:23<00:00, 151.00it/s]


In [120]:
revised_main['421,0001881832']

[{'url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=421&aid=0001881832',
  'title': '국회, 北 장거리 미사일 발사 규탄..',
  'office': '뉴스1',
  'type': 'text',
  'expose': '02/10 14:55 ~ 02/11 07:46 (16시간50분) 노출',
  'date': datetime.datetime(2016, 2, 11, 0, 0)},
 {'url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=421&aid=0001881832',
  'title': '국회, 北 장거리 미사일 발사 규탄 ..',
  'office': '뉴스1',
  'type': 'text',
  'expose': '02/10 14:48 ~ 02/10 14:55 (6분) 노출',
  'date': datetime.datetime(2016, 2, 11, 0, 0)},
 {'url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=421&aid=0001881832',
  'title': '국회, 北 장거리 미사일 발사 규탄..',
  'office': '뉴스1',
  'type': 'text',
  'expose': '02/10 14:55 ~ 02/11 07:46 (16시간50분) 노출',
  'date': datetime.datetime(2016, 2, 12, 0, 0)}]

In [105]:
import copy

'''
여러 개의 main_info가 나오는데 그를 날짜 순으로 정렬하여 가장 처음과 나중의 정보만 남길 수 있도록 설정함.
'''

def simplify(info):
    if not info[0].get('expose'):
        
    exposes = sorted(set(list(map(lambda x: x['expose'], info))))
    errors = []
    year = str(info[0]['date'].year)
    for st in exposes:
        time = st[st.index('(') + 1: st.index(')') - 1]
        if time == '0':
            errors.append(st)
    exposes = [elem for elem in exposes if elem not in errors]
    
    start_times = []
    end_times = []

    for st in exposes:
        st_splited = st.split(' ')
        start_time = datetime.strptime(year + '/' + st_splited[0] + ' ' + st_splited[1], '%Y/%m/%d %H:%M')
        end_time = datetime.strptime(year + '/' + st_splited[3] + ' ' + st_splited[4], '%Y/%m/%d %H:%M')
        start_times.append(start_time)
        end_times.append(end_time)
        
    try:
        first = min(start_times)
        last = max(end_times)
    except:
        return None
    
    
    result = copy.deepcopy(info[0])
    result['expose_first'] = first
    result['expose_last'] = last
    del result['date']
    del result['expose']
    
    return result


# k = simplify(revised_main['421,0001881832'])
s = {k: simplify(v) for k,v in revised_main.items()}

In [13]:
# 기사 업로드 시간 컬럼 더하기
spt_path = '../news data/articles/20200316_comment_split/'
spt_list = os.listdir(spt_path)[1:]

def uploaded (aid, afile):
    info = afile.get(aid)
    if not info:
        return None
    else:
        return info['timestamp']
for day in tqdm(['20151208.csv']):
    df = pd.read_csv(spt_path + day, index_col=False, dtype = object)
    with open(naver_path + day[:8]+'.json', 'r') as f:
        article_file = json.load(f)
    df['naver_uploaded'] = df.apply(lambda x: uploaded(x['naver_id'], article_file), axis = 1)
    df.to_csv('../news data/articles/20200316_comment_split/' + day[:8] + '.csv', index = False)

100%|██████████| 1/1 [00:01<00:00,  1.49s/it]


In [14]:
naver_main_list.index('20151208.json')

587

# 댓글을 메인 등재 전/중/후로 분류

In [3]:
naver_path = '../news data/articles/all_daily_articles_naver/'
naver_list = os.listdir(naver_path)
print('NAVER FROM {} to {}'.format(naver_list[0], naver_list[-1]))

with open('../pilgrim data/sid_to_cat.csv', 'r', encoding = 'utf-8') as g:
    reader = csv.reader(g)
    sid_to_cat = {line[0]: line[1] for line in reader}
    del sid_to_cat['\ufeffsid']

naver_main_path = '../news data/main/main_naver/main/'
naver_main_list = os.listdir(naver_main_path)
idx = naver_main_list.index('2014-04-30.json')
print(idx)
naver_main_list_original = naver_main_list[idx:]
naver_main_list = list(map(lambda x: x[:4] + x[5:7] + x[8:], naver_main_list_original))
print('NAVER MAIN FROM {} to {}'.format(naver_main_list[0], naver_main_list[-1]))

naver_comment_path = '../news data/comments/daily_comments/'
naver_comment_list = os.listdir(naver_comment_path)[:-1]
print('NAVER COMMENT FROM {} to {}'.format(naver_comment_list[0], naver_comment_list[-1]))

NAVER FROM 20000202.json to 20191002.json
1788
NAVER MAIN FROM 20140430.json to 20190404.json
NAVER COMMENT FROM .DS_Store to 20190705.json


In [4]:
# 각 댓글을 메인 등재 시간 이전, 메인 등재 중, 메인 등재 이후의 시간에 나누어 카운트함.
def comment_split (df, comments):
    for aid in df.naver_id.tolist():
        before_main = 0
        during_main = 0
        after_main = 0

        first_exposed = datetime.datetime.fromisoformat(df.loc[df.naver_id == aid].naver_main_exposed_first.item())
        last_exposed = datetime.datetime.fromisoformat(df.loc[df.naver_id == aid].naver_main_exposed_last.item())

        try:
            target_coms = list(map(lambda x: datetime.datetime.fromisoformat
                               (x['createdAt'][:-2]+':'+x['createdAt'][-2:]).replace(tzinfo = None)
                               ,comments[aid]))
        except:
            df.loc[df.naver_id == aid, 'before_main'] = before_main
            df.loc[df.naver_id == aid, 'during_main'] = during_main
            df.loc[df.naver_id == aid, 'after_main'] = after_main
            df.loc[df.naver_id == aid, 'naver_commentCount_new'] = before_main + during_main + after_main
            continue

        for comment_time in target_coms:
            if comment_time <= first_exposed:
                before_main += 1
            elif comment_time <= last_exposed:
                during_main += 1
            else:
                after_main += 1
                
#         assert before_main + during_main + after_main == len(target_coms)
                
        df.loc[df.naver_id == aid, 'before_main'] = before_main
        df.loc[df.naver_id == aid, 'during_main'] = during_main
        df.loc[df.naver_id == aid, 'after_main'] = after_main
        df.loc[df.naver_id == aid, 'naver_commentCount_new'] = before_main + during_main + after_main
        
    df.before_main = df.before_main.astype('int')
    df.during_main = df.during_main.astype('int')
    df.after_main = df.after_main.astype('int')
    df.naver_commentCount_new = df.naver_commentCount_new.fillna(0)
    df.naver_commentCount_new = df.naver_commentCount_new.astype('int')
    
    return df

In [10]:
# 같은 알고리즘을 다음 코멘트 정보에 대해서도 적용함.
def comment_split_daum (df, comments):
    for i in range(len(df)):
        aid = df.loc[i].daum_id
        before_main = 0
        during_main = 0
        after_main = 0

        first_exposed = datetime.datetime.fromisoformat(df.loc[i].naver_main_exposed_first)
        last_exposed = datetime.datetime.fromisoformat(df.loc[i].naver_main_exposed_last)

        try:
            target_coms = list(map(lambda x: datetime.datetime.fromisoformat
                               (x['createdAt'][:-2]+':'+x['createdAt'][-2:]).replace(tzinfo = None)
                               ,comments[aid]))
        except:
            df.loc[i, 'before_main_daum'] = before_main
            df.loc[i, 'during_main_daum'] = during_main
            df.loc[i, 'after_main_daum'] = after_main
            continue

        for comment_time in target_coms:
            if comment_time <= first_exposed:
                before_main += 1
            elif comment_time <= last_exposed:
                during_main += 1
            else:
                after_main += 1
                
        df.loc[i, 'before_main_daum'] = before_main
        df.loc[i, 'during_main_daum'] = during_main
        df.loc[i, 'after_main_daum'] = after_main
        
    df.before_main_daum = df.before_main_daum.astype('int')
    df.during_main_daum = df.during_main_daum.astype('int')
    df.after_main_daum = df.after_main_daum.astype('int')
    
    return df

In [12]:
daum_comment_path = '../news data/comments/daum/'
spt_path = '../news data/articles/20200325_comment_split/'
spt_list = os.listdir(spt_path)

for day in tqdm(spt_list):
    df = pd.read_csv(spt_path + day, index_col=False, dtype = object)
    with open(daum_comment_path + day[:8] + '.json', 'r') as f:
        comment_file = json.load(f)
        
    df = comment_split_daum(df, comment_file)
    df.to_csv('../news data/articles/20200326_comment_split/' + day[:8] + '.csv', index = False)

# 1시간, 8시간, 24시간 별로 댓글 수 확인해서 첨부

In [11]:
# 메인에서 빠진 이후 1시간, 8시간, 24시간 지났을 때의 댓글 추이를 확인하는 코드.
def comment_split_by_time (df, comments):
    for i in range(len(df)):
        first_exposed = datetime.datetime.fromisoformat(df.loc[i].naver_main_exposed_first)
        last_exposed = datetime.datetime.fromisoformat(df.loc[i].naver_main_exposed_last)
        aid = df.loc[i]['naver_id']
        
        in_1hour = 0
        in_8hour = 0
        in_24hour = 0

        try:
            target_coms = list(map(lambda x: datetime.datetime.fromisoformat
                               (x['createdAt'][:-2]+':'+x['createdAt'][-2:]).replace(tzinfo = None)
                               ,comments[aid]))
        except:
            df.loc[i, 'in_1hour_naver'] = in_1hour
            df.loc[i, 'in_8hour_naver'] = in_8hour
            df.loc[i, 'in_24hour_naver'] = in_24hour
            continue

        for comment_time in target_coms:
            if comment_time >= last_exposed:
                hour_diff = int((comment_time - last_exposed).total_seconds() // 3600)
                if hour_diff == 0:
                    in_1hour += 1
                if hour_diff < 8:
                    in_8hour += 1
                if hour_diff < 24:
                    in_24hour += 1
                
        df.loc[i, 'in_1hour_naver'] = in_1hour
        df.loc[i, 'in_8hour_naver'] = in_8hour
        df.loc[i, 'in_24hour_naver'] = in_24hour
        
    df.in_1hour_naver = df.in_1hour_naver.astype('int')
    df.in_8hour_naver = df.in_8hour_naver.astype('int')
    df.in_24hour_naver = df.in_24hour_naver.astype('int')
    
    return df

In [14]:
spt_path = '../news data/articles/20200324_comment_split/'
spt_list = os.listdir(spt_path)[1:]
for day in tqdm(spt_list):
    df = pd.read_csv(spt_path + day, index_col=False, dtype = object)
    with open(naver_comment_path + day[:8] + '.json', 'r') as f:
        comment_file = json.load(f)
    df = comment_split_by_time(df, comment_file)
    cols = df.columns.tolist()
    cols = cols[:19] + cols[-3:] + cols[19:-3]
    df = df[cols]
    df.to_csv('../news data/articles/20200325_comment_split/' + day, index = False)

In [22]:
# 위와 동일한 알고리즘을 다음 파일에 대해서도 적용하였음.
def comment_split_by_time_daum (df, comments):
    for i in range(len(df)):
        first_exposed = datetime.datetime.fromisoformat(df.loc[i].naver_main_exposed_first)
        last_exposed = datetime.datetime.fromisoformat(df.loc[i].naver_main_exposed_last)
        aid = df.loc[i]['daum_id']
        
        in_1hour_daum = 0
        in_8hour_daum = 0
        in_24hour_daum = 0
        daum_totalCount = 0

        try:
            target_coms = list(map(lambda x: datetime.datetime.fromisoformat
                               (x['createdAt'][:-2]+':'+x['createdAt'][-2:]).replace(tzinfo = None)
                               ,comments[aid]))
        except:
            df.loc[i, 'in_1hour_daum'] = in_1hour_daum
            df.loc[i, 'in_8hour_daum'] = in_8hour_daum
            df.loc[i, 'in_24hour_daum'] = in_24hour_daum
            df.loc[i, 'daum_totalCount'] = daum_totalCount
            continue

        for comment_time in target_coms:
            if comment_time >= last_exposed:
                hour_diff = int((comment_time - last_exposed).total_seconds() // 3600)
                if hour_diff == 0:
                    in_1hour_daum += 1
                if hour_diff < 8:
                    in_8hour_daum += 1
                if hour_diff < 24:
                    in_24hour_daum += 1
                    
            daum_totalCount += 1
                
        df.loc[i, 'in_1hour_daum'] = in_1hour_daum
        df.loc[i, 'in_8hour_daum'] = in_8hour_daum
        df.loc[i, 'in_24hour_daum'] = in_24hour_daum
        df.loc[i, 'daum_totalCount'] = daum_totalCount
        
    df.in_1hour_daum = df.in_1hour_daum.astype('int')
    df.in_8hour_daum = df.in_8hour_daum.astype('int')
    df.in_24hour_daum = df.in_24hour_daum.astype('int')
    df.daum_totalCount = df.daum_totalCount.astype('int')
    
    return df

In [23]:
# 전체 파일에 대해 적용.
spt_path = '../news data/articles/20200316_comment_split/'
spt_list = os.listdir(spt_path)
daum_comment_path = '../news data/comments/daum/'
idx = spt_list.index('20180101.csv')
spt_list = spt_list[idx:]
for day in tqdm(spt_list):
    df = pd.read_csv(spt_path + day, index_col=False, dtype = object)
    with open(daum_comment_path + day[:8] + '.json', 'r') as f:
        comment_file = json.load(f)
    df = comment_split_by_time_daum(df, comment_file)
    df.to_csv('../news data/articles/20200319_comment_split/' + day, index = False)